### GTA Lab 3: Python and PostgreSQL - Solution

In [ ]:
import fiona
from shapely.geometry import LineString
import psycopg2
from psycopg2.extensions import AsIs
import pandas as pd

In [ ]:
shpfile=fiona.open("swissTLM3D_TLM_EISENBAHN.shp","r")
print(shpfile.schema)
len(list(shpfile))
print(len(shpfile[28]))
print(shpfile[345])
print(shpfile[345]["geometry"]["coordinates"])

In [ ]:
shxfile=fiona.open("swissTLM3D_TLM_EISENBAHN.shx","r")
print(shxfile.schema)
#ich glaub das ist genau das gleiche, also enthält die gleichen Informationen aber vielleicht in einem anderen Format==> shp/shx

In [ ]:
#Connecting with the database:
import numpy as np

structure=(list(shpfile))
print(structure)
db_gta = {"dbname": "gta",
          "port": "5432",
          "user": "gta_p4",
          "password": "***REMOVED***",
          "host": "ikgpgis.ethz.ch"}
conn = psycopg2.connect(**db_gta)
cur=conn.cursor()
for i in range(len(list(shpfile))):
    line = LineString(np.array(structure[i]['geometry']['coordinates'])[:,:2])
    #print(np.array((structure[i]['geometry']['coordinates']))[:][0:2])
    #print(line)

    cur.execute(f"INSERT INTO train_tracks (geom) VALUES ('{line}');")

conn.commit()
conn.close()

In [ ]:

sample=np.array([(2696490.9499999993, 1261351.914999999, 438.5939999999973), (2696504.1799999997, 1261368.3150000013, 438.5939999999973), (2696511.1000000015, 1261377.484000001, 438.5939999999973)])
print(sample)
line = LineString(sample[:,0:2])
print(line)


## Introduction:

Today we aim to transfer data from two different data sources into a common database. Both datasets describe artworks in Zurich city (e.g. a statue). The data is provided in two .csv files.


### Import modules

In the beginning of a Python scripts we import the libraries that are required for executing the script. Libraries provide Python code that was written by other people to solve a specific problem. If you have a task in Python, it is always worth researching for existing modules, since there are many well-documented open source packages available, e.g. in the Python Package Index (PyPI). 

Today we will mainly use the package **psycopg2**. This library provides functionality to communicate in Python with a Postgresql database. 

In [ ]:
import psycopg2
from psycopg2.extensions import AsIs
import pandas as pd

## Exercise 1: Working with Pandas and csv files

### General information about csv files

CSV is one of the most common formats to save tabular data. CSV files have a fixed number of columns and each element in a column is separated from the next element with a certain separator, e.g. a comma. However, a practical challenge in working with csv files is the missing standard of their format: Different separators may be used (comma, semicolon, etc) or different encodings are user (e.g. ASCII or UTF-8). The format must therefore be defined by the user when reading and writing CSV files.

#### Identifying csv parameters

In order to identify these parameters, you can for example open the csv file in a simple text editor and visually identify the used separators. To do so, it is important to use an editor that shows the file in its most raw format (for example NOT excel, because it would already render the file and not show the raw text). On Windows you can for example use Notepad or Notepad++, on Mac you can use TextEdit. 

#### Reading csv files with Python

In order to load the csv data and to write it to a PostgeSQL database, you first need to read the csv file. You can use the pandas function **pd.read_csv**. The function takes a file path as input and all further optional parameters are used to read the file in the correct format. You call find the documentation with all parameters here: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html. 

Important parameters are the following:

  + **sep**: The separator used in the csv for separating columns
  + **thousands**: Separator for thousands, e.g. 1.000 or 1,000 
  + **decimal**: Separator for decimals (default is a dot)
  + **encoding**: Encoding of the csv file. It is one of the most common error sources when reading csv files. **Here are some popular endocings that can be tested:**: 'latin-1', 'utf-8', 'iso-8859-1', 'utf_8_sig' or 'cp1252'

Example:
```python
data = pd.read_csv(path_csv, sep=",", decimal=",") 
```

### Exercise 1.1: Reading csv files

Load the files **kunstimstadtraum_source1.csv** and **kunstimstadtraum_source2.csv** which you can find in the `data` folder. 
- First open the data in a text editor and check the formatting. Which separator is used?
- Use the function pd.read_csv as described above and set the parameters correctly. Which encoding is used? Do you have an idea why UTF-8 is not used?
- Check the correctness by printing the first few rows of the table with the function **df.head()** (if df is your Pandas dataframe) 

In [ ]:
# Read kunstimstadtraum_source1.csv
source1 = pd.read_csv("data/kunstimstadtraum_source1.csv", encoding = "latin-1", sep=";")

In [ ]:
# Check if everything was read correctly
source1.head()

In [ ]:
# Read kunstimstadtraum_source2.csv
source2 = pd.read_csv("data/kunstimstadtraum_source2.csv", encoding = "latin-1", delimiter="|")

In [ ]:
# Check if everything was read correctly
source2.head()

### Task 1.2: Working with Pandas:

For selecting and manipulating data you can use SQL, if the data is already available in a database. On the other hand, data can also be manipulated directly in Python. For more complex processing, for connecting with other data, data processing in Python is very important. We therefore practice data selection and manipulation with Pandas in this lab. 

Pandas represents tables in so-called DataFrames. The DataFrame class implements many methods for table operations, such as selection, sorting, grouping or joining.

Selecting single columns is done with a syntax similar to a dictionary:

```python
source1["GATTUNG"]

0      Freiplastik
1      Freiplastik
2          Brunnen
3          Brunnen
4      Freiplastik
          ...     
189     Gestaltung
190    Freiplastik
191    Freiplastik
192     Bauplastik
193        Brunnen
Name: GATTUNG, Length: 194, dtype: object
```

### Exercise 1.2.1: Sort the artworks in table `source1` alphabetically by the first name of the artist.

To do so, check out the documentation of the function sort_values: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sort_values.html

In [ ]:
# TODO: sort values
df_sorted = source1.sort_values(by="KUENSTLER")
df_sorted

### Task 1.2.2: Selection

Create a new table `df_filtered`, that should only contain artworks of type (Gattung) "Brunnen". 

Tip: As shown in the lab slides, we can filter with boolean indices. The Syntax is 

```
df_filtered = df[df[column_name] == value]
```

In [ ]:
# TODO: filter for Gattung Brunnen
df_filtered = source1[source1["GATTUNG"]=="Brunnen"]
df_filtered

### Exercise 1.3:  Iterating over dataframes

In exercise 3 we will insert all rows into the database individually. As a preparation, we want to use a pandas function to iterate over the rows of a DataFrame, namely `df.iterrows()`. This function is a *generator*, which means that you can access the elements with a for-loop. In each iteration (for each row), we can again select a column as above. This yields a single value of one particular table cell. **Print the artist (KUENSTLER) of the first five rows sequentially in a for loop:**

In [ ]:
# TODO: print rows with iterrows
for i, row in source1.iterrows():
    print(row["KUENSTLER"])
    if i>4:
        break

# PostgreSQL and psycopg2

We now want to operate on a PostgreSQL database (with PostGIS extension) with Python. This enables you to keep large amounts of data in a database, while making use of the rich functionality of Python for preprocessing and analysis. We use the library **psycopg2** for communicating with the database.

#### Accessing a database with psycopg2

In order to connect to a database, you require several information about the database, such as database name, user name, password, port, host. These variables usually stay the same, so it makes sense to save them in variables in the beginning. 

To conntect to a database, you can use the **connect** method, which returns a **connection** object. The syntax is the following: 

```python
conn = psycopg2.connect(dbname=my_dbname, port=my_port, user=my_user, password=my_password, host=my_host)
```

**Tip:** The syntax later can be simplified significantly if we store the information in a Python dictionary and use the [*kwargs* syntax](https://realpython.com/python-kwargs-and-args/):
```python
db_credentials = {"dbname": NAME,
                  "port": PORTNUMMER,
                  "user": BENUTZERNAME,
                  "password": PASSWORT,
                  "host": HOSTNUMMER}
conn = psycopg2.connect(**db_credentials)
```

#### Connection and Cursor objekts

The variable `conn` is now an object that maintains and administers the connection to your database. To interact with the database now, you require a cursor. A cursor is created with the command
```
conn.cursor()
```

In today's lab you will mainly use the following cursor methods:

+ **cur.execute("sql string")**: Executes any sql query on your database.
+ **conn.commit()**: Submits the changes to your database (such that it is permanently modified).
+ **conn.rollback()**: Reverts non-permanent changes. This is in particular necessary if you have commited an invalid SQL request. You first have to revert it with conn.rollback() before continuing in that case.
+ **conn.close()**: Closes the connection

Here is a typical sequence of commands for a database request:

```python
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()
cur.execute("your_sql_query")
conn.commit()
conn.close()
```

For further information on psycopg, see http://initd.org/psycopg/docs/usage.html

# Exercise 2: Creating a new table with psycopg2

### Preliminaries:
+ schema name gtaXX (your schema)
+ table name: kunst_im_stadtraum
+ the table should have the following fields:

|field name | datatyp|
|-----------|---------|
|id         | SERIAL PRIMARY KEY|
|titel      | VARCHAR|
|kuenstler  | VARCHAR|
|standort   | VARCHAR|
|lon        | FLOAT|
|lat        | FLOAT|
|geometry   | POINT|

### Procedure:

1. Save the information of your database connection (host, username, etc, see above) in a dictionary
2. Connect to the database
3. Initialize the cursor
4. Create a SQL command as a Python string, where you use the CREATE TABLE statement in order to create a table with all fields **except for the geomety** (we will deal with the geometry later). In order to **create a table inside your schema** (e.g. gta23_01), write the table name with the following syntax: `schema_name.table_name` (e.g. gta23_01.kunst_im_stadtraum_01).

Syntax of **CREATE TABLE** statement:
```SQL
CREATE TABLE IF NOT EXISTS schema_name.table_name(
column1 datatype, column2 datatype, ...., columnX datatype);
```
5. Execute the SQL query
6. Add the Geometry column with a new SQL query. 

Syntax of **AddGeometryColumn** statement (see documentation http://postgis.net/docs/AddGeometryColumn.html):
```SQL
SELECT AddGeometryColumn(varchar schema_name, varchar table_name, 
varchar column_name, integer SRID, varchar geom_type, integer dimension)
```

7. Validate the changes with the commit command
8. Close the database connection

### Tips: 
+ Write the commands in separate cells. This simplifies debugging.
+ If you have sent an invalid request to the database, you have to reset the connection object. There are two possibilities to do so:
    + Reset the erronous command with ```conn.rollback()```
    + Connect again from scratch (```conn = psycopg2.connect(**db_credentials)```)
+ **Important:** The varchar arguments have to be passed as a string in the SQL query. However, since the whole SQL request is a String itself, we have to use a String inside a String, i.e.: "some outer string 'some inner string'"

### Steps 1-3

In [ ]:
# SOLUTION
db_credentials = {
    "user": "gta_tutor",
    "password": "gta_pw",
    "host": "ikgpgis.ethz.ch",
    "port": "5432",
    "dbname": "gta"
}

# establish database connection
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

### Step 4

In [ ]:
# TODO: query string
create_table_query = "CREATE TABLE IF NOT EXISTS gta_tutor.kunst_im_stadtraum\
(id SERIAL PRIMARY KEY, titel VARCHAR, kuenstler VARCHAR, standort VARCHAR, lon FLOAT, lat FLOAT)"

### Step 5

In [ ]:
# TODO: execute
cur.execute(create_table_query)

### Step 6

In [ ]:
# TODO: add geometry
add_geom_query = "SELECT AddGeometryColumn('gta_tutor', 'kunst_im_stadtraum', 'geometry', 4326, 'POINT', 2)"

### Step 7

In [ ]:
# TODO: execute
cur.execute(add_geom_query)

### Step 8 - 9

In [ ]:
# TODO: finalize and close connection
conn.commit()
conn.close()

#### Check in PgAdmin whether your table was created correctly!

# Exercise 3: Import data

## Exercise 3.1: Importing the first csv file

Fill the fields of your newly created table with the rows of the first csv file ("data/kunstimstadtraum_source1.csv" / source1), with the following steps:

1. Create a database connection and a cursor
2. Check that your input data looks correct by printing `source1.head()`
3. Use the method `cur.mogrify` to test a String with varying content. cur.mogrify() provides you the string which would then be sent to the database with cur.execute. Check the psycopg2 syntax here: [https://www.psycopg.org/docs/usage.html](https://www.psycopg.org/docs/usage.html)
4. Iterate over the `source` DataFrame with a for-loop
5. In each iteration, set suitable variables with the necessary values of the table row. For example, we need the value for column `KUENSTLER` in order to fill the varchar field `kuenstler` of the table in our database, so we can save it in a variable `kuenstler = row["KUENSTLER"]` as an intermediate step. **Import the WGS representation of latitude and longitude**!
6. Execute a SQL-INSERT statement with cur.execute, where the variables of step 5 are used in the SQL query string. Syntax of **INSERT** statement:

```SQL
INSERT INTO schema_name.table_name (column1, column2, column3, ...)
VALUES (value1, value2, value3, ...); 
```

7. Validate your changes with the commit command and close the database connection.
8. Test your your result with the function **count_items**, which counts the number of rows in your table in the PostgreSQL database. After reading the first .csv file, there should be 194 rows.

### Important: Do NOT fill the geom-column at this point! We will add the geometry with PostGIS in exercise 4.

### Step 1

In [ ]:
# TODO: establish database connection
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

### Step 2

In [ ]:
# TODO: check source1 table
source1[:3]

### Step 3

Example for using placeholders in a String:

cur.mogrify("INSERT INTO test (num, data) VALUES (%s, %s)",(100, "abc'def"))


In [ ]:
#This is the example from above:
sql_string_with_placeholders = "INSERT INTO test (num, data) VALUES (%s, %s)"
cur.mogrify(sql_string_with_placeholders,(100, "abc'def"))


### Step 4 - 6

In [ ]:
#loop all rows from csv1
for idx, row in source1.iterrows():
    
    #prepate list of input values
    titel = row["TITEL"]
    kuenstler = row["KUENSTLER"]
    standort = row["STANDORT"]
    lon = row["Easting_WGS"]
    lat = row["Northing_WGS"]
    
    # prepare insert string
    insert_string = """INSERT INTO gta_tutor.kunst_im_stadtraum (titel, kuenstler, standort, lon, lat)
                    VALUES(%s, %s, %s, %s, %s);"""
    # cur.execute() statement
    cur.execute(insert_string, (titel, kuenstler, standort, lon, lat))
    

### Step 7

In [ ]:
#validate process
conn.commit()
conn.close()

### Definition der Funktion count_items

In [ ]:
def count_items(tablename, dbname, port, user, password, host):
    """count all items in a database """
    
    conn = psycopg2.connect(dbname=dbname, port=port, user=user, password=password, host=host)
    cur = conn.cursor()
    cur.execute("select count(*) from %s", (AsIs(tablename),))
    conn.commit()

    results = cur.fetchall()
    conn.close()

    return results

### Step 8

Test your result with the function **count_items**.

In [ ]:
# Test your output with count_items functions
items_in_table = count_items("kunst_im_stadtraum", **db_credentials)

In [ ]:
# Check whether the number of rows in the database corresponds to the rows in our source1 table
print(f"In the database we now have {items_in_table[0][0]} items")
assert items_in_table[0][0] == len(source1)

## Exercise 3.2: Importing the second csv file

This exercise is almost identical to 3.1. You can reuse your code of exercise 3.1. **However, the columns are named differently in the second csv! Make sure to use the correct column names.**

Again, make sure to only fill the columns titel, kuenstler, standort, lon and lat, since we will fill the geom column in exercise 4.

Use the function **count_items** again to check your result. There should now be 388 rows in your table in the database.

### Step 1-6

In [ ]:
source2

In [ ]:
# TODO: Connect and iterate over the rows of source2 to import them
# solution
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

#loop all rows from csv1
for idx, row in source2.iterrows():
    
    #prepate list of input values
    titel = row["titel"]
    kuenstler = row["kuenstler"]
    standort = row["standort"]
    lon = row["lon"]
    lat = row["lat"]
    
    # prepare insert string
    insert_string = """INSERT INTO gta_tutor.kunst_im_stadtraum (titel, kuenstler, standort, lon, lat)
                    VALUES(%s, %s, %s, %s, %s);"""
    # cur.execute() statement
    cur.execute(insert_string, (titel, kuenstler, standort, lon, lat))
    
#validate process
conn.commit()
conn.close()

### Step 7

In [ ]:
# TODO: count rows with count_items function
items_in_table = count_items("kunst_im_stadtraum", **db_credentials)

In [ ]:
print(f"In the database we now have {items_in_table[0][0]} items")
assert items_in_table[0][0] == 388

## Exercise 3.3: Reading data from the database

To check our table and what we have inserted, we can print a few rows from the database, using a simple SQL SELECT statement. When we execute a query with `cur.execute()` and `cur.commit()`, we can retrieve the results with `cur.fetchall()`. 

### 3.3.1: Add a suitable sql query to the following code, such that the first five rows of the table in the PostgreSQL database are printed

In [ ]:
conn = psycopg2.connect(**db_credentials)
cur = conn.cursor()

# SOLUTION
sql = "SELECT * from gta_tutor.kunst_im_stadtraum LIMIT 5"

cur.execute(sql)
conn.commit()
results = cur.fetchall()
conn.close()
print(results)

You may have realized that we copy and paste the code to initialize and to close connections a lot of times. Since we generally want to avoid repitions when writing code, we will now define a function `read_sql`, which allows to execute any SQL query on the database:

Example:
```python
data = read_sql(sql_string, **db_credentials)
```

In [ ]:
def read_sql(sql, dbname, port, user, password, host ):
    """execute sql query on database and return results"""

    conn = psycopg2.connect(dbname=dbname, port=port, user=user, password=password, host=host)
    cur = conn.cursor()
    
    cur.execute(sql)
    conn.commit()

    #get results and clean up
    results = cur.fetchall()
    conn.close()
    return results

### 3.3.2 Filtering data with SQL via Python:

Use the function **read_sql** for the following query: Find all artworks that were created by Franz Fischer (kuenstler = 'Franz Fischer (1900-1980)').

In [ ]:
# get all art pieces on fountains
# SOLUTION
sql = "SELECT titel, kuenstler, standort FROM gta_tutor.kunst_im_stadtraum WHERE kuenstler = 'Franz Fischer (1900-1980)'"
data = read_sql(sql, **db_credentials)
data

# Exercise 4: Creating a geometry

You have loaded the data from both csv files into the PostGIS database. However, the column `geometry` is still empty. In this exercise, we aim to fill this column with `Point` data.

### Steps:

1. Create a similar function as `read_sql` which can be used to write to the database. Name this function `write_sql`. The function should take an sql query and the database credentials as inputs. It should then execute the query and close the connection.

2. Create an SQL string for adding a geometry. Here, we will use the PostGIS function `ST_MakePoint`, and fill it with the values from the already existing columns `lat` and `lon`. The MakePoint statement consists of several components: The core is an *UPDATE* statement, because we want to change the existing column `geometry`.

+ Syntax of **Update** statement
```SQL
UPDATE schema_name.table_name SET column_name = value
```

Then you need to create a point with *ST_MakePoint* and add it with a coordinate reference system using *ST_SetSRID*. Example:

```SQL
UPDATE schema_name.table_name SET geometry_column_name = 
ST_SetSRID(ST_MakePoint(lon_column_name,lat_column_name),4326);
```

3. Execute the SQL string with your new `write_sql` function.
4. Print the first five lines of your table again to check whether the geometry column was added successfully.

### Step 1:

In [ ]:
# SOLUTION
def write_sql(sql, dbname, port, user, password, host):
    """execute sql query on database"""
    conn = psycopg2.connect(dbname=dbname, port=port, user=user, password=password, host=host)
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()
    conn.close()

### Step 2:

In [ ]:
# SOLUTION
geom_sql = "UPDATE gta_tutor.kunst_im_stadtraum SET geometry = ST_SetSRID(ST_MakePoint(lon,lat),4326);"

### Step 3

In [ ]:
# SOLUTION
write_sql(geom_sql, **db_credentials)

### Step 4

In [ ]:
# SOLUTION
read_sql("SELECT * from gta_tutor.kunst_im_stadtraum LIMIT 5", **db_credentials)

# Exercise 5: Spatial queries with PostGIS

So far, we have only inserted a geometry, but we have not executed any spatial queries. As seen in the SQL part of the exercise, PostGIS provides many functions for filtering and joining geometric data. Use the function `ST_DWithin` to answer the following query:

**Find all artworks, that are less than 1km away from the artwork [Steinvase auf Sockel]**

Tips:
* [Steinvase auf Sockel] is located at (8.58148801, 47.348233)
* The function `ST_DWithin` must be used in a WHERE Statement. Check out the documentation: https://postgis.net/docs/ST_DWithin.html
* Careful: We want to know the distance in meters! 


In [ ]:
# SOLUTION:
# We set the argument `use_squeriod` to true, to ensure that the distance in meters is returned.
geom_query = "SELECT * FROM gta_tutor.kunst_im_stadtraum WHERE ST_DWithin(geometry, 'SRID=4326;POINT(8.58148801 47.348233)', 1000, true)"
results = read_sql(geom_query, **db_credentials)

In [ ]:
assert len(results) == 5

## Exercise 6: Simplified database access with pandas

So far, we have used psycopg2 functions to get to know the typical workflow of database access. However, there are of course Python packages that have already integrated database access and take over the task of dealing with the connection object. Pandas offers this functionality with the [to_sql](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html) and [read_sql](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html) functions, and Geopandas has the corresponding functions `to_postgis` and `read_postgis`. For further work with databases in your project, you can use these functions to query data from the database or to add / update tables. **As an exercise, have a look at the documentation of `read_postgis` and use it to retrieve the data (first 10 rows) including the geometry from our PostGIS database.**

Tip: Pandas requires a connection engine created with sqlalchemy instead of psycopg2. Use the following line to set up the engine:

Tip2: If you receive the error `TypeError: __init__() got multiple values for argument 'schema'` you need to update pandas.

In [ ]:
from sqlalchemy import create_engine
import geopandas as gpd
import numpy as np

In [ ]:
db_credentials

In [ ]:
create_string = f"postgresql://{db_credentials['user']}:{db_credentials['password']}@{db_credentials['host']}:{db_credentials['port']}/{db_credentials['dbname']}"        
print("Create string for sqlalchemy engine:", create_string)
engine = create_engine(create_string)

In [ ]:
# TODO: use gpd.read_postgis to retrieve the first 10 rows including the geometry
# SOLUTION:
test = gpd.read_postgis("SELECT * FROM kunst_im_stadtraum LIMIT 10", con=engine.connect(), geom_col="geometry")

In [ ]:
# assert that the geometry is read correctly
assert np.isclose(test.geometry.iloc[0].x, 8.58148801000993), "wrong point, check your geometry and rows"